In [ ]:
### Install requirements
from os.path import isfile

repository   = "https://github.com/lmingari/olot-course.git"
requirements = "requirements-section2-2.txt"

if not isfile(requirements):
    !git clone {repository}
    %cd olot-course
    !pip install -r {requirements}

# 2.2 A multilayer perceptron (MLP) for classification
***

* We propose a simple model for assessing the __impact of tephra fallout__ at a given location in La Palma based on deposit thickness data

* We use the volcanic tephra deposition dataset from the 2021 Tajogaite Eruption on La Palma reported by [Shatto et al. (2024)][dataset]

* The dataset comprises a total of 415 in-situ field measurements sampled across the Island of La Palma along with 66 values estimated from surface changes in areas not suitable for in-situ sampling

* The dataset is available on [Zenodo][zenodo]

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S2352340923009800-gr2.jpg" width="400">

[dataset]: https://doi.org/10.1016/j.dib.2023.109949
[zenodo]: https://doi.org/10.5281/zenodo.8338991

#### Definition of impact classes

We define three classes of tephra fallout impacts in terms of the deposit thickness:

|Impact    | Thickness range   | Class |
|----------|:-----------------:|:-----:|
| Low      | < 1 mm            | 0     |
| Moderate | 1 - 100 mm        | 1     |
| High     | > 100 mm          | 2     |

> __Goal:__
>
>  Train a MLP to "predict" the class of impact for a given coordinate (lat, lon)

## Main routines for training and evaluation

#### Importing modules

In [ ]:
import pandas as pd                                        # tools for data manipulation and analysis
import numpy as np                                         # numerical operations on arrays
import matplotlib.pyplot as plt                            # plots and visualizations

import torch                                               # main PyTorch library for tensor computation
import torch.nn as nn                                      # building blocks for creating and training neural networks
import torch.optim as optim                                # implementation of various optimization algorithms
from torch.utils.data import Dataset, DataLoader           # dataset utilities

from sklearn.model_selection import train_test_split       # utility to easily split datasets

#### Training routine

In [ ]:
def train_epoch(model, loader, criterion, optimizer):
    """
    Performs one complete training epoch over the dataset.
    
    Args:
        model: The neural network model to be trained
                
        loader: DataLoader that provides batches of training data. 
            Each iteration yields a batch of inputs and targets
        
        criterion: Loss function used to compute the training loss (e.g., nn.MSELoss).
            Takes model predictions and targets as input
        
        optimizer: Optimization algorithm used to update model parameters (e.g., Adam, SGD).
    
    Returns:
        Returns training metrics (average loss and accuracy for the epoch)
    """
    
    # Set training mode
    model.train()
    
    total_loss = 0.0
    corrects   = 0

    # Mini-batch loop
    for xb, yb in loader:
        # Model prediction
        logits = model(xb)
        # Compute loss
        loss = criterion(logits, yb)

        # Update gradients
        optimizer.zero_grad()        
        loss.backward()

        # Update parameters
        optimizer.step()

        # Predict category
        predicted_category = torch.argmax(logits,dim=1)

        # Update metrics
        total_loss += loss.item()
        corrects += (predicted_category == yb).sum().item()

    # Return average loss and accuracy (%)
    return total_loss / len(loader.dataset), 100 * corrects / len(loader.dataset)

#### Inference routine

In [ ]:
def evaluate_epoch(model, loader, criterion):
    # Set inference mode
    model.eval()
    
    total_loss = 0.0
    corrects = 0
    
    with torch.no_grad():
        for xb, yb in loader:
            logits = model(xb)
            loss = criterion(logits, yb)

            predicted_category = torch.argmax(logits,dim=1)

            # Update metrics
            total_loss += loss.item()
            corrects += (predicted_category == yb).sum().item()

    # Return average loss and accuracy (%)
    return total_loss / len(loader.dataset), 100 * corrects / len(loader.dataset)

## Training and validation datasets

#### Loading data

In [ ]:
df = pd.read_csv('data/palma-deposit.csv')
df

#### Defining the target

In [ ]:
## Define the categories

df['category'] = 0
df.loc[df['thickness_cm']>0.1,'category'] = 1
df.loc[df['thickness_cm']>10,'category']  = 2
df

In [ ]:
## Check dataset balance
_ = df['category'].value_counts().plot(kind='bar')

#### Plotting the distribution of measurements

In [ ]:
from helper_plot import create_map

fig, ax = create_map()

for target, label, color in [(0,'Low','b'), (1,'Moderate','y'), (2,'High','r')]:
    df.loc[df.category==target].plot.scatter(
        x='lon', y='lat', 
        c=color,
        s=4,
        alpha=0.5,
        label = label,
        ax = ax)

#### Splitting datasets

In [ ]:
# Define features and target
features = ['lat','lon']
X = df[features].values
y = df['category'].values

# Splitting arrays (80/20 %)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Size of the training dataset: ", X_train.shape)
print("Size of the validation dataset: ", X_val.shape)

#### Dataset objects

In [ ]:
from helper import Standardize, ThicknessDataset

# Compute normalization stats from training data only
mean = X_train.mean(axis=0)
std  = X_train.std(axis=0)

# Define transform
transform = Standardize(mean,std)

# Datasets
dataset_train = ThicknessDataset(X_train, y_train, transform=transform)
dataset_val   = ThicknessDataset(X_val, y_val, transform=transform)

## Model arquitecture

In [ ]:
# ---------------------------
# Simple MLP classifier
# ---------------------------
class Classifier(nn.Module):
    def __init__(self, in_dim=2, hidden=64, out_dim=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)  # output logits
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)  # shape [B]

## Training loop

In [ ]:
#
# Configuration
#
conf = {
    'BATCH_SIZE': 16,
    'LEARNING_RATE': 4E-4,
    'NUM_EPOCHS': 200,
}

# DataLoader
loader_train = DataLoader(dataset_train, batch_size=conf['BATCH_SIZE'], shuffle=True)
loader_val   = DataLoader(dataset_val,   batch_size=conf['BATCH_SIZE'], shuffle=False)

# Model
model = Classifier()

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=conf['LEARNING_RATE'])

In [ ]:
train_losses = []
train_accs   = []
val_losses   = []
val_accs     = []

for epoch in range(conf['NUM_EPOCHS']):
    train_loss, train_acc = train_epoch(model, loader_train, criterion, optimizer)
    val_loss,   val_acc   = evaluate_epoch(model, loader_val, criterion)

    # Store current losses/accuracies
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    # Print log
    if epoch%10 == 0:
        print(f"Epoch {epoch+1:03d}:")
        print(f"Train loss (accuracy): {train_loss:.4f} ({train_acc:.2f}%) || Validation loss (accuracy): {val_loss:.4f} ({val_acc:.2f}%)")

In [ ]:
fig, axs = plt.subplots(ncols = 2, figsize=(12,5))

axs[0].plot(train_losses, label = 'Training loss')
axs[0].plot(val_losses,   label = 'Validation loss')

axs[1].plot(train_accs, label = "Training accuracy")
axs[1].plot(val_accs,   label = "Validation accuracy")

axs[0].set(ylabel = 'Average loss', xlabel = 'Epoch')
axs[1].set(ylabel = 'Accuracy (%)', xlabel = 'Epoch')

for ax in axs: ax.legend()

## Decision regions

In [ ]:
from helper_plot import plot_decision_regions

fig, ax = plot_decision_regions(model, transform)

for category, label, color in [(0,'Low','b'), (1,'Moderate','y'), (2,'High','r')]:
    df.loc[df.category==category].plot.scatter(
        x = 'lon', y = 'lat', 
        c     = color,
        s     = 5,
        alpha = 0.5,
        label = label,
        ax = ax)

fig.set_size_inches(6, 8)

> &#9998; **Exercise 1:** <br>
> * Redefine the model class `Classifier` by removing the ReLU activation functions
> * How are the decision regions modified?

> &#9998; **Exercise 2:** <br>
> Increase overfitting by:
> - Decreasing the learning rate ($\eta = 10^{-4}$)
> - Increasing the number of epochs ($N=1200$)
> - Adding more neurons or more layers